# Load Environment & Libraries

In [13]:
#!pip install --upgrade pip
#!pip install --upgrade langchain
#!pip install docarray
#!pip install python-dotenv
#!pip install tiktoken
#pip install pypdf
#!pip install openai
#!pip install streamlit

In [2]:
import os
# from dotenv import load_dotenv
# load_dotenv() # read local .env file
secret_key = os.environ['OPENAI_API_KEY']

from pathlib import Path
Path.cwd()

PosixPath('/Users/samuelchazy/ML_files_X/Applied_Projects/Experiments/LLM_Apps/notebooks')

In [3]:
from langchain.document_loaders import PyPDFLoader,TextLoader
from langchain.text_splitter import TokenTextSplitter,RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA,LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.prompts import ChatPromptTemplate,PromptTemplate
from IPython.display import display, Markdown

#### Load file

In [25]:
file = '/Users/samuelchazy/ML_files_X/Applied_Projects/Experiments/Generic_templates/LLM_templates/data/AI_in_education.pdf'

pdf_loader_1 = PyPDFLoader(file)
document_1_loader = pdf_loader_1.load()

In [26]:
# my_database_string = ''
# for i in range(len(document_1_loader)):
#     section = document_1_loader[i].page_content.strip().split('\n')
#     my_database_string += "".join(section)

Created a chunk of size 838, which is longer than the specified 500
Created a chunk of size 903, which is longer than the specified 500
Created a chunk of size 1050, which is longer than the specified 500
Created a chunk of size 517, which is longer than the specified 500
Created a chunk of size 814, which is longer than the specified 500
Created a chunk of size 1061, which is longer than the specified 500
Created a chunk of size 520, which is longer than the specified 500
Created a chunk of size 610, which is longer than the specified 500
Created a chunk of size 717, which is longer than the specified 500
Created a chunk of size 811, which is longer than the specified 500
Created a chunk of size 551, which is longer than the specified 500
Created a chunk of size 666, which is longer than the specified 500
Created a chunk of size 808, which is longer than the specified 500
Created a chunk of size 986, which is longer than the specified 500
Created a chunk of size 1008, which is longer 

In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=150,chunk_overlap=20)
text = text_splitter.create_documents([str(document_1_loader)])

In [14]:
llm = ChatOpenAI(temperature = 0.0,model='gpt-3.5-turbo')
embeddings = OpenAIEmbeddings()

In [44]:
my_database = DocArrayInMemorySearch.from_documents(document_1_loader, embeddings)
retriever = my_database.as_retriever()
retriever

VectorStoreRetriever(vectorstore=<langchain.vectorstores.docarray.in_memory.DocArrayInMemorySearch object at 0x13f28a750>, search_type='similarity', search_kwargs={})

# RetrievalQA from chain type

In [23]:
def input_query():
    question = input('What is your query?')
    display(f'Query: {question}')
    return question

In [24]:
def run_model(question,doc_retriever):

    response_template = """
    You are a professional in the Educational Field.
    Your task is to read research papers, research documents, educational journals, and conference papers.
    You should be analytical and reply in depth.
    Always reply in a polite and professional manner.

    Divide your answer when possible into bullet points style:
    • What is the question.
    • How did you analyze the question.
    • What did you read in the document to analyze the question.
    • What is your answer to the question.
    • What made you come up with this conclusion or reply.
    • Add the references and citations related to this question and answer from this document in bullet points each one separately.

    If you don't know the answer to the question, then reply by: "I can't be confident about my answer because I am missing the context or some information!"

    <Input>
    {context}

    <Query>
    Question: {question}

    <Output>
    Answer:
    """

    prompt_response_style = "reply in a calm and respectful tone"

    prompt = PromptTemplate(template=response_template, input_variables=["context", "question"])
    chain_type_kwargs = {'prompt':prompt}

    query_model = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=doc_retriever,
        chain_type_kwargs=chain_type_kwargs,
    verbose=False)
    chat_response = query_model.run(question)

    # prompt = ChatPromptTemplate.from_template(template=response_template)
    # chat_messages = prompt.format_messages(style=prompt_response_style, text=text, format_instructions=response_template)
    # chat_response = llm(chat_messages).content

    return chat_response

'Query: What are the use cases of AI in education?'

The use cases of AI in education can be summarized as follows:

- Automating and tracking learner's progress: AI has the potential to automate and help track the learner's progress in various skills, identifying where a human teacher's assistance is needed.
- Identifying effective teaching methods: AI can be used to help identify the most effective teaching methods based on students' contexts and learning background.
- Automating operational tasks: AI can automate monotonous operational tasks, generate assessments, and automate grading and feedback.
- Recommending personalized learning experiences: AI can recommend personalized learning experiences to students based on their contexts and learning backgrounds.
- Enhancing assessments: AI can enhance our understanding of learners, not only in terms of what they know but also how they learn and which pedagogies work for them.
- Providing intelligent tutoring systems: AI can provide intelligent learning environments that can interact with 

In [17]:
query = input_query()
model = run_model(query,retriever)
print(model)

In [ ]:
# What are the use cases of AI in education?